In [16]:
import subprocess
import os
import glob
import sys
import time
from read_metadata_HIWI import read_metadata
import subprocess
from run_inference_HIWI import run_inference, extract_frames, video_to_frames, get_gpu_memory
from export_tracks import export_tracks, calc_distance, get_color
from detections_to_tracks import detections_to_tracks
import pandas as pd 
import tqdm
import numpy as np
import shutil
import datetime as dt

import os
from decord import VideoReader, cpu, gpu
import cv2
import subprocess as sp
import tqdm

import torch 
torch.cuda.empty_cache()

#from numba import cuda

In [17]:
### this is the only necessary input

# tracks file
tracks_file = 'C:/Users/meerkat/Documents/social_sleep/DATA/tracking_output/fewer_tracks_parameters/20190817/tracks_20190817_190002669000-final-final-final-final-final-final.npy'


In [18]:
# provide a file path for the temporary images that will be created from the videos
images_folder = "C:/Users/meerkat/Documents/social_sleep/DATA/frames_for_finalizing/"

# where you want the tracking video to end up
output_folder = 'C:/Users/meerkat/Documents/social_sleep/DATA/tracking_output/finalized_tracks/'

In [19]:
vid_name  = tracks_file.split( "/")[ -1 ].split( "_", 1 )[ 1 ].split( "." )[ 0 ]

vid_name

'20190817_190002669000-final-final-final-final-final-final'

In [20]:
# save the name of the original video. This will be necessary to pull in the .txt file from the server
og_vid_name = vid_name.split( '-' )[ 0 ]

# save the night this video came from
start_timestamp = dt.datetime.strptime( og_vid_name, '%Y%m%d_%H%M%S%f' )

start_timestamp - dt.timedelta( hours = 12 )

night = dt.datetime.strftime( start_timestamp, '%Y%m%d' )

In [21]:
# input video
input_video_file = os.path.join( 'Z:/baboon/archive/processed/video/thermal/2019_summer/cliff_data/mp4_files/viewpoint_1/T1020/', night, str( og_vid_name + '.mp4' ) )



In [22]:
input_video_file

'C:/Users/meerkat/Downloads/20190817\\20190817_190002669000.mp4'

In [23]:
## what do these do?
# %store vid_name
# %store -r vid_name

# metadata = read_metadata( vid_name ) ## expanded below

### the only thing actually needed from this whole cell is the bit that sets the folder location where the videos for tracking are (metadata.folder_data_server)
class struct:
    pass

## instantiate metadata
metadata = struct()

## this is the repository -- the main folder the project is occuring within
#metadata.folder_main = "C:/Users/meerkat/Documents/social_sleep/"

## this will use a folder that has all the images that will be turned into the video
metadata.folder_images = images_folder

## this will make a folder with the output of the video
metadata.folder_output = os.path.join( output_folder, night )

## I don't know what this is for
#metadata.folder_annotations =  metadata.folder_data +  'annotations/'




In [24]:
###### extracting frames from input video #######
frames_already_extracted = False
delete_frames = True

overwrite = True 

# enter the interval of frames from the input videos that you would like to extract (interval = 1 means each frame will be used for tracking)
image_interval = 1 ## image_interval is the step length between each frame to extract from the video. every = 1 pulls out every frame. every = 2 pulls out every other frame, etc.

## set the start and end frames to be extracted from the input video. This makes it so we can limit the frames on which we do tracking to a certain part of the video. -1 means the whole video
start= -1
end= -1

##### Plotting tracks hyperparameters #####
steps = 10  # 90 # determines the number of timesteps into the past (with each frame = 1 timestep) the tail should be printed
step_inc = 1 # 5 # determines the timestep interval at which a point for the tail is actually printed
circle_radius = 3 # 20
interpolation_factor = 10
with_tail = True
sample_range = range(1,1) # if you set 'sample_range' equal to this, it will actually go through all frames of the input video

os.makedirs( ( os.path.join( metadata.folder_output ) ), exist_ok=True )

video_out_file_name = "video_" + vid_name + ".mp4" 

## assign the name of the video file for the video with the tracks
video_out_file = os.path.join(metadata.folder_output, video_out_file_name)

#### the following lines of code extract all the frames from the input video

if frames_already_extracted != True:

    ## not really sure what the os.path.join does here, but this just sets up the folder where the images will be extracted to
    os.makedirs( ( os.path.join( metadata.folder_images ) ), exist_ok=True )

    assert os.path.exists( input_video_file )  # assert the video file exists

    # load the VideoReader
    vr = VideoReader( input_video_file, ctx = cpu(0) )  # can set to cpu or gpu .. ctx=gpu(0)

    if start < 0:  # if start isn't specified lets assume 0
        start = 0
    if end < 0:  # if end isn't specified assume the end of the video
        end = len(vr)

    ## you can change every to be image_interval everywhere below if you want
    every = image_interval

    ## make a list of the frame indices that we will pull out from the video (this is every frame when every = 1 )
    frames_list = list(range(start, end, every ) )

    ## start a counter of the number of images that are saved (why can't this be inferred from the length of frames_list?)
    saved_count = 0

    if every > 50 and len(frames_list) < 1000:  # this is faster for every > 25 frames and can fit in memory

        frames = vr.get_batch(frames_list).asnumpy() ## pull out all the relevant frames as numpy arrays, in a single operation 

        for index, frame in zip(frames_list, frames):  # loop through the frames. For each frame...

            save_path = os.path.join( metadata.folder_images, "{:06d}.jpg".format(index))  # create the save path. This will save each frame as a JPG with a 6 digit number as a file name (which indicates which frame in the original video it was) in the image folder input above

            if not os.path.exists(save_path) or overwrite:  # if it doesn't exist or we want to overwrite anyways

                cv2.imwrite(save_path, cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))  # save the extracted image

                saved_count += 1  # increment our counter by one

    else:  # this is faster for every <25 and consumes small memory

        try:

            for index in frames_list:  # lets loop through the frames until the end

                ## why can't the line above just read: for index in frames_list: ?? (it was for index in range( start, end )  Then we don't have to set the if statement below

                frame = vr[ index ]  # read an image from the capture

                # this shouldn't be needad anymore with my adjustment above... if index % every == 0:  # if this is a frame we want to write out based on the 'every' argument

                save_path = os.path.join( metadata.folder_images, "{:06d}.jpg".format(index))  # create the save path. This will save each frame as a JPG with a 6 digit number as a file name (which indicates which frame in the original video it was) in the image folder input above

                if not os.path.exists( save_path ) or overwrite:  # if it doesn't exist or we want to overwrite anyways

                    cv2.imwrite(save_path, cv2.cvtColor(frame.asnumpy(), cv2.COLOR_RGB2BGR))  # save the extracted image

                    saved_count += 1  # increment our counter by one

        except:

            frames = vr.get_batch(frames_list).asnumpy() ## pull out all the relevant frames as numpy arrays, in a single operation 

            for index, frame in zip(frames_list, frames):  # loop through the frames. For each frame...

                save_path = os.path.join( metadata.folder_images, "{:06d}.jpg".format(index))  # create the save path. This will save each frame as a JPG with a 6 digit number as a file name (which indicates which frame in the original video it was) in the image folder input above

                if not os.path.exists(save_path) or overwrite:  # if it doesn't exist or we want to overwrite anyways

                    cv2.imwrite(save_path, cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))  # save the extracted image

                    saved_count += 1  # increment our counter by one

    print( saved_count )  # and return the count of the images we saved

# training

5742


In [25]:
from IPython import get_ipython
import pandas as pd, numpy as np, cv2, os, glob, matplotlib.pyplot as plt
from matplotlib import cm
from read_metadata_HIWI import read_metadata
from tqdm import tqdm
from scipy import interpolate
from scipy.ndimage import gaussian_filter1d

os.path.join(metadata.folder_images, '*.jpg') # I don't think this does anything

frame_files = sorted( glob.glob( os.path.join( metadata.folder_images, '*.jpg' ) ) ) # save the names of the frame files that were extracted from the input video above

tracks = np.load( tracks_file, allow_pickle=True ) # load the tracks that we just saved above

im = cv2.imread(frame_files[0]) # read in the first frame of the video

fourcc = (cv2.VideoWriter_fourcc)(*'mp4v') # declare the codec for the video

video_writer = cv2.VideoWriter( video_out_file, fourcc, 23.976, (im.shape[1], im.shape[0]) ) # instantiate the video at the desired file location with the codec declared above. Pull the dimensions for the video from the first frame extracted from the input video

colors = np.random.uniform(size=(len(tracks))) # generate random numbers between 0 and 1 (with uniform probability). There will be as many numbers as there are tracks

colors = [get_color(c) for c in colors] # turn each number into a color

all_distance = [] # len(frame_files) # instantiate an empty list

if len(sample_range) == 0: # if the sample_range has been set to the default (i.e. essentially hasn't been defined)

    sample_range = range(1, len(frame_files), 1) # set the range equal to the entire input video

for frame_ind in tqdm(sample_range): # loop through the frames of the input video. For each frame...

    im = cv2.imread(frame_files[frame_ind]) # read the frame in using cv2

    for track_ind, track in enumerate(tracks): # loop through the tracks. For each track...

        rel_frame = frame_ind - track['first_frame'] # find the number of this frame compared to the frame on which the track starts from

        if rel_frame >= 0 and track['last_frame'] > frame_ind: # if the current frame comes after the track has started and before the track has ended...

            center_pos = track['track'][rel_frame] # find and save the position of the track on this frame

            if not np.isnan( np.sum( center_pos ) ):                        

                cv2.circle(im, (int(center_pos[0]), int(center_pos[1])), circle_radius, colors[track_ind], -1) # print a circle on the frame representing the center position of the current detection (i.e. the location of the track at this frame)
                cv2.putText(im, str(track_ind), (int(center_pos[0] + 10 ), int(center_pos[1] + 10 )), 0, 1, colors[track_ind],3) # label the location of the current detection of the track with the index of the track

                if with_tail: # if there should be a tail printed on the frame showing where the track had been in previous frames...

                    if rel_frame > steps: # if the number of frames from the start of the track to the current frame are greater than the number of steps to include in the tail...

                        pos = track['track'][ range( rel_frame - steps, rel_frame - step_inc, step_inc) ] # find the position of the track at each time for which a tail point should be included (this is determined by steps, which determines how many timesteps back (with each frame = 1 time step) the tail should represent, and step_inc, which determines the interval of timesteps for which a tail point is actually printed)

                        interp_pos = np.zeros((len(range(step_inc, len(pos)*interpolation_factor-interpolation_factor, 1)),2)) # sets up a two column array that will be filled with the interpolation tail points to print

                        interp = interpolate.interp1d(range(step_inc, len(pos)*interpolation_factor, step_inc*interpolation_factor), pos[:,0], kind = "linear") # do a linear interpolation on the x coordinates of the previous positions of this track to be used in the tail that were extracted above

                        interp_pos[:,0] = gaussian_filter1d(interp(range(step_inc, len(pos)*interpolation_factor-interpolation_factor, 1)),interpolation_factor) # set the x coordinates within the array to be printed equal to the linear interpolation (with some guassian filtering?)

                        interp = interpolate.interp1d(range(step_inc, len(pos)*interpolation_factor, step_inc*interpolation_factor), pos[:,1], kind = "linear") # do a linear interpolation on the y coordinates of the previous positions of this track to be used in the tail that were extracted above

                        interp_pos[:,1] = gaussian_filter1d(interp(range(step_inc, len(pos)*interpolation_factor-interpolation_factor, 1)),interpolation_factor) # set the y coordinates within the array to be printed equal to the linear interpolation (with some guassian filtering?)

                        size_vec = np.power(2,np.linspace(0.5,circle_radius,len(range(0,len(interp_pos),1)))) # make an array of the size that each point in the tail should be printed as, with earlier points being printed smaller

                        for inc in range(0,len(interp_pos),1): # loop through the indices of the coordinates of the tail points. For each tail point...

                            pos_loc = interp_pos[ inc ] # extract the coordinate of the point

                            distance = calc_distance(pos_loc, center_pos) # calculate the distance between this tail point and the current location of the track
                            # if distance < circle_radius:
                            #     continue
                            if not np.isnan( np.sum( pos_loc ) ):

                                cv2.circle(im, (int(pos_loc[0]), int(pos_loc[1])), int(np.round(size_vec[inc])), colors[track_ind], -1) # print the tail point at these coordinates, with appropriate size according to that calculated above

    video_writer.write( im ) # write the printed image with the tracks on it to the video

video_writer.release() # release the video



files_in_directory = os.listdir(metadata.folder_images) # make a list of the image files of the frames from the input video

filtered_files = [file for file in files_in_directory if file.endswith('.jpg')] # filter these to make sure they are the images (i.e. end with a 'jpg')

if delete_frames == True:

    ## delete the image files of the frames of the input video from the folder
    for file in filtered_files:
        path_to_file = os.path.join(metadata.folder_images, file)
        os.remove(path_to_file)

100%|██████████| 5741/5741 [19:55<00:00,  4.80it/s]


In [26]:
## turn the npy tracks into a dataframe and save the dataframe in the finalized tracks folder as a csv

all_tracking = pd.DataFrame({'frame':[],  'id':[],  'x':[],  'y':[] }) # start a dataframe that will be filled with all the location of all the tracks for the video

all_distance = [] # len(frame_files) # instantiate an empty list

In [27]:
for track_ind, track in enumerate( tracks ):
    
    track_frames = list( range( track['first_frame'], track['last_frame'] + 1 ) )
    
    to_append = pd.DataFrame( {'frame': pd.Series( track_frames ), 'id': [ track_ind ] * len( track_frames ), 'x': pd.Series( track['track'][:,0] ) ,  'y': pd.Series( track['track'][:,1] ) } )    

    all_tracking = all_tracking.append( to_append, ignore_index = True )
    
all_tracking[ 'vid_name' ] =  og_vid_name

all_tracking = all_tracking.astype( { 'frame': int, 'id': int, 'x': float, 'y': float } )

all_tracking.to_csv( os.path.join( metadata.folder_output, 'tracks_' + vid_name + '.csv' ), index=False) # write the csv with all the tracks to a file


In [28]:
## now copy the tracks file and the .txt file for the night over to the finalized tracks folder

In [29]:
current_txt = os.path.join( 'Z:/baboon/archive/processed/video/thermal/2019_summer/cliff_data/mp4_files/viewpoint_1/T1020/', night, og_vid_name + '.txt' )

In [15]:
## copy associated txt file into the finalized tracks folder
shutil.copy( current_txt, metadata.folder_output )

FileNotFoundError: [Errno 2] No such file or directory: 'Z:/baboon/archive/processed/video/thermal/2019_summer/cliff_data/mp4_files/viewpoint_1/T1020/20190815\\20190815_163505281000.txt'

In [30]:
## copy the tracks file into the finalized tracks folder
shutil.copy( tracks_file, metadata.folder_output )

'C:/Users/meerkat/Documents/thermal_baboon/RESULTS/tracking_output/viewpoint_1/finalized_tracks/20190817\\tracks_20190817_190002669000-final-final-final-final-final-final.npy'